In [1]:
import sys
import os

# Insert the parent directory of "app" into sys.path
# so that Python recognizes "app" as an importable package.
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
sys.path.append(parent_dir)

In [2]:
from dotenv import load_dotenv
load_dotenv()  # This looks for .env in the current working directory

True

In [3]:
import time

import pandas as pd
import seaborn as sns
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
)

from langchain_core.messages import HumanMessage, ToolMessage, AIMessage


In [4]:
# Benchmark data
test_df = pd.read_csv('benchmark.csv', delimiter=";")
test_df.head()

,question,ground_truth
0,What are some contracts that ended after 1990-...,There are a total of 377 contracts that ended ...
1,What are some contracts that ended after 1990-...,"There are 48 contracts classified as ""Strategi..."
2,"What are some contracts with HC2 Holdings, Inc.?","There is one contract involving HC2 Holdings, ..."
3,Which year was the most contracts signed?,The year with the most contracts signed was 20...
4,Which company has the most active contracts?,The company with the most active contracts is ...


In [5]:
from backend.src.agent import get_agent

In [6]:
agent = get_agent()

In [7]:
input_messages = [HumanMessage(content="Which year was the most contracts signed?")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which year was the most contracts signed?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (9e5e79b8-cb4f-4ce4-8a7d-148f16d77937)
 Call ID: 9e5e79b8-cb4f-4ce4-8a7d-148f16d77937
  Args:
    cypher_aggregation: RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1
================================= Tool Message =================================
Name: ContractSearch

[{"year": 2024, "count": 1}]
================================== Ai Message ==================================

In 2024, the most contracts were signed, with a total of 1 contract.


In [8]:
# History
messages

{'messages': [HumanMessage(content='Which year was the most contracts signed?', additional_kwargs={}, response_metadata={}, id='a9856d0c-b183-4423-a217-499ef40197d9'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'ContractSearch', 'arguments': '{"cypher_aggregation": "RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-694d13d8-87b3-4f7a-9f5e-975194447a0d-0', tool_calls=[{'name': 'ContractSearch', 'args': {'cypher_aggregation': 'RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1'}, 'id': '9e5e79b8-cb4f-4ce4-8a7d-148f16d77937', 'type': 'tool_call'}], usage_metadata={'input_tokens': 562, 'output_tokens': 26, 'total_tokens': 588, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='[{"year": 2024, "count": 1}]', name='ContractSearch', id='ae728e1a-2c4b-

In [9]:
# Chat with history
input_messages = messages['messages'] + [HumanMessage(content="List all contracts where a party from Japan is involved.")]
messages = agent.invoke({"messages": input_messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Which year was the most contracts signed?
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (9e5e79b8-cb4f-4ce4-8a7d-148f16d77937)
 Call ID: 9e5e79b8-cb4f-4ce4-8a7d-148f16d77937
  Args:
    cypher_aggregation: RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1
================================= Tool Message =================================
Name: ContractSearch

[{"year": 2024, "count": 1}]
================================== Ai Message ==================================

In 2024, the most contracts were signed, with a total of 1 contract.
================================ Human Message =================================

List all contracts where a party from Japan is involved.
================================== Ai Message ==================================
Tool Calls:
  ContractSearch (cfc3eb30-fa29-45a6-814f-aa798cf3ac

In [13]:
def get_answer(input: str) -> str:
    input_messages = [HumanMessage(content=input)]
    messages = agent.invoke({"messages": input_messages})
    tools = [m.additional_kwargs.get('function_call') for m in messages["messages"] if isinstance(m, AIMessage) and m.additional_kwargs.get('function_call')]
    answer = messages["messages"][-1].content
    return answer, tools

In [14]:
get_answer("List all contracts where a party from Japan is involved.")

('I found 2 contracts that involve a party from Japan. One is a Manufacturing contract between Photronics, Inc., Dai Nippon Printing Co., Ltd., Photronics DNP Photomask Corporation, and Xiamen American Japan Photronics Mask Co., Ltd. The other is a Maintenance contract regarding the Asia Pacific Cable Network 2 (APCN 2) with a multitude of parties including Japan Telecom Co., Ltd., KDD Corporation, and NTT Communications Corporation.',
 [{'name': 'ContractSearch', 'arguments': '{"parties": ["Japan"]}'}])

In [15]:
from tqdm import tqdm

# Get workflows generated answers
results = []
latencies = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing questions"):
    question = row['question']
    start = time.time()
    try:
        data = get_answer(question)
    except:
        data = {"answer": "timeout", "question": question}
    end = time.time()
    latencies.append(end - start)
    results.append(data)

test_df["latencies"] = latencies
test_df["response"] = [el[0] for el in results]
test_df["tools"] = [el[1] for el in results]

Processing questions: 100%|██████████| 45/45 [01:02<00:00,  1.39s/it]


In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Adjust width to prevent wrapping
pd.set_option('display.max_colwidth', None)  # Prevents truncation of long column values

In [18]:
test_df[['response','question', 'tools']]

,response,question,tools
0,"Here are some contracts that ended after 1990-01-01. I am showing you a sample of 5 contracts out of 377 total contracts. The sample includes the contract ID, scope, type, countries, effective and end dates, file ID, parties involved, and a summary of the contract.",What are some contracts that ended after 1990-01-01?,"[{'name': 'ContractSearch', 'arguments': '{""min_end_date"": ""1990-01-01""}'}]"
1,"Here are some strategic alliance contracts that ended after 1990-01-01. I have included the contract ID, the parties involved, the effective date, and the end date:\n\n* **Contract ID**: ELFBEAUTY,INC\_07\_02\_2020-EX-10.1-COOPERATION AGREEMENT, **Parties**: e.l.f. Beauty, Inc. and Marathon Partners Equity Management, LLC, **Effective Date**: 2020-7-1, **End Date**: 2021-11-1\n* **Contract ID**: HC2HOLDINGS,INC\_05\_14\_2020-EX-10.1-COOPERATION AGREEMENT, **Parties**: MG Capital Management Ltd., Percy Rockdale LLC, Rio Royal LLC, HC2 Holdings, Inc., **Effective Date**: 2020-5-13, **End Date**: 2021-1-1\n* **Contract ID**: ROCKYMOUNTAINCHOCOLATEFACTORY,INC\_12\_23\_2019-EX-10.2-STRATEGIC ALLIANCE AGREEMENT, **Parties**: Farids & Co. LLC, Edible Arrangements, LLC, Rocky Mountain Chocolate Factory, Inc., **Effective Date**: 2019-12-20, **End Date**: 2021-12-20\n* **Contract ID**: BERKELEYLIGHTS,INC\_06\_26\_2020-EX-10.12-COLLABORATION AGREEMENT, **Parties**: Ginkgo Bioworks, Inc., Berkeley Lights, Inc., **Effective Date**: 2019-9-13, **End Date**: 2026-9-30\n* **Contract ID**: INTELLIGENTHIGHWAYSOLUTIONS,INC\_01\_18\_2018-EX-10.1-Strategic Alliance Agreement, **Parties**: Bravatek Solutions, Inc., Fazync LLC, **Effective Date**: 2018-1-10, **End Date**: 2019-1-10",What are some contracts that ended after 1990-01-01 that fall under strategic alliance?,"[{'name': 'ContractSearch', 'arguments': '{""contract_type"": ""Strategic Alliance"", ""min_end_date"": ""1990-01-01""}'}]"
2,"I found one strategic alliance contract involving HC2 Holdings, Inc. with MG Capital Management Ltd., Percy Rockdale LLC, and Rio Royal LLC. This contract, effective from May 13, 2020, to January 1, 2021, focuses on modifying the composition of HC2 Holdings, Inc.'s board of directors and other related matters. The monetary value associated with the contract is $352,290.25.","What are some contracts with HC2 Holdings, Inc.?","[{'name': 'ContractSearch', 'arguments': '{""parties"": [""HC2 Holdings, Inc.""]}'}]"
3,"The year with the most contracts signed was 2024, with a total of 1 contract.",Which year was the most contracts signed?,"[{'name': 'ContractSearch', 'arguments': '{""cypher_aggregation"": ""RETURN effective_date.year AS year, count(*) AS count ORDER BY year DESC LIMIT 1""}'}]"
4,"The company with the most active contracts is HOF Village, LLC, with a total of 2 active contracts.",Which company has the most active contracts?,"[{'name': 'ContractSearch', 'arguments': '{""cypher_aggregation"": ""\nUNWIND parties AS party\nWITH party, active, count(*) AS contract_count\nWHERE active = true\nRETURN party, contract_count\nORDER BY contract_count DESC\nLIMIT 1""}'}]"
5,I do not have the ability to search for specific terms within the contract. I am able to search the summary of the contract. Would you like me to search for contracts containing the word 'indemnification' in the summary?,Do we have any contracts about indemnification?,[]
6,"To assist you with your request, I need to know how you would like to define total value. Please provide more details.",Which Hosting contracts have the highest total value?,[]
7,Microgenics Corporation is listed as a party in 1 contract.,How many contracts does Microgenics Corporation have?,"[{'name': 'ContractSearch', 'arguments': '{""parties"": [""Microgenics Corporation""]}'}]"
8,I can search for contracts by country. Would you like me to search for contracts with the country code 'US'?,What contracts are governed by US law?,[]
9,"The party with the most Strategic Alliance contracts is Vertical

In [19]:
dataset = Dataset.from_pandas(test_df)

result = evaluate(
    dataset,
    metrics=[
        answer_correctness
    ],
)

Evaluating:   0%|          | 0/45 [00:00<?, ?it/s]

Exception raised in Job[7]: KeyError('reference')
Exception raised in Job[8]: KeyError('reference')
Exception raised in Job[12]: KeyError('reference')
Exception raised in Job[9]: KeyError('reference')
Exception raised in Job[13]: KeyError('reference')
Exception raised in Job[14]: KeyError('reference')
Exception raised in Job[6]: KeyError('reference')
Exception raised in Job[16]: KeyError('reference')
Exception raised in Job[11]: KeyError('reference')
Exception raised in Job[15]: KeyError('reference')
Exception raised in Job[10]: KeyError('reference')
Exception raised in Job[22]: KeyError('reference')
Exception raised in Job[20]: KeyError('reference')
Exception raised in Job[25]: KeyError('reference')
Exception raised in Job[23]: KeyError('reference')
Exception raised in Job[18]: KeyError('reference')
Exception raised in Job[26]: KeyError('reference')
Exception raised in Job[24]: KeyError('reference')
Exception raised in Job[27]: KeyError('reference')
Exception raised in Job[28]: KeyErr

In [20]:
result.to_pandas()

,user_input,response,reference,answer_correctness
0,What are some contracts that ended after 1990-01-01?,"Here are some contracts that ended after 1990-01-01. I am showing you a sample of 5 contracts out of 377 total contracts. The sample includes the contract ID, scope, type, countries, effective and end dates, file ID, parties involved, and a summary of the contract.","There are a total of 377 contracts that ended after January 1, 1990",0.352932
1,What are some contracts that ended after 1990-01-01 that fall under strategic alliance?,"Here are some strategic alliance contracts that ended after 1990-01-01. I have included the contract ID, the parties involved, the effective date, and the end date:\n\n* **Contract ID**: ELFBEAUTY,INC\_07\_02\_2020-EX-10.1-COOPERATION AGREEMENT, **Parties**: e.l.f. Beauty, Inc. and Marathon Partners Equity Management, LLC, **Effective Date**: 2020-7-1, **End Date**: 2021-11-1\n* **Contract ID**: HC2HOLDINGS,INC\_05\_14\_2020-EX-10.1-COOPERATION AGREEMENT, **Parties**: MG Capital Management Ltd., Percy Rockdale LLC, Rio Royal LLC, HC2 Holdings, Inc., **Effective Date**: 2020-5-13, **End Date**: 2021-1-1\n* **Contract ID**: ROCKYMOUNTAINCHOCOLATEFACTORY,INC\_12\_23\_2019-EX-10.2-STRATEGIC ALLIANCE AGREEMENT, **Parties**: Farids & Co. LLC, Edible Arrangements, LLC, Rocky Mountain Chocolate Factory, Inc., **Effective Date**: 2019-12-20, **End Date**: 2021-12-20\n* **Contract ID**: BERKELEYLIGHTS,INC\_06\_26\_2020-EX-10.12-COLLABORATION AGREEMENT, **Parties**: Ginkgo Bioworks, Inc., Berkeley Lights, Inc., **Effective Date**: 2019-9-13, **End Date**: 2026-9-30\n* **Contract ID**: INTELLIGENTHIGHWAYSOLUTIONS,INC\_01\_18\_2018-EX-10.1-Strategic Alliance Agreement, **Parties**: Bravatek Solutions, Inc., Fazync LLC, **Effective Date**: 2018-1-10, **End Date**: 2019-1-10","There are 48 contracts classified as ""Strategic Alliance"" that ended after January 1, 1990.",0.287626
2,"What are some contracts with HC2 Holdings, Inc.?","I found one strategic alliance contract involving HC2 Holdings, Inc. with MG Capital Management Ltd., Percy Rockdale LLC, and Rio Royal LLC. This contract, effective from May 13, 2020, to January 1, 2021, focuses on modifying the composition of HC2 Holdings, Inc.'s board of directors and other related matters. The monetary value associated with the contract is $352,290.25.","There is one contract involving HC2 Holdings, Inc. This contract is a ""Cooperation Agreement"" under the type ""Strategic Alliance."" It was effective from May 13, 2020, and ended on January 1, 2021. The contract's scope involved an agreement to modify the composition of the company's board of directors and related matters.",0.502307
3,Which year was the most contracts signed?,"The year with the most contracts signed was 2024, with a total of 1 contract.","The year with the most contracts signed was 2020, with a total of 50 contracts.",0.232140
4,Which company has the most active contracts?,"The company with the most active contracts is HOF Village, LLC, with a total of 2 active contracts.","The company with the most active contracts is HOF Village, LLC, with a total of 2 active contracts.",1.000000
5,Do we have any contracts about indemnification?,I do not have the ability to search for specific terms within the contract. I am able to search the summary of the contract. Would you like me to search for contracts containing the word 'indemnification' in the summary?,"There are currently no contracts in our system that specifically mention ""indemnification"" in their summaries. If you have any more specific criteria or need assistance with something else, feel free to ask!\n",0.476060
6,Which Hosting contracts have the highest total value?,"To assist you with your request, I need to know how you would like to define total value. Please provide more details.",NaN,NaN
7,How many contracts does Microgenics Corporation have?,Microgenics Corporation is listed as a party in 1 contract.,NaN,NaN
8,What contracts are go